**Our first step is to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:**

Let's begin by importing our libraries.  I made the choice to use BeautifulSoup to scrape the Wikipedia table due to its flexibility and my expectations for future utilization in more complex scraping.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


I loaded the website into the wiki_url variable and reviewed the parsed html by printing the values with prettify.  For presentation purposes, I've commented out the print function due to the large amount of data produced but this was reviewed to properly understand the table I wanted to extract.

In [2]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_url,"lxml")
#print(soup.prettify())

In [3]:
table = soup.find('table')
df = pd.read_html(str(table))[0]
df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Clean up our dataframe by removing all Not assigned Boroughs.  We will also merge any common postal codes and include all respective neighborhoods in the same row. 

In [7]:
df=df.rename(columns = {'Neighbourhood':'Neighborhood'})

# Get names of invalid Boroughs and Neighborhoods for which column 'Not assigned'
notassigned_boro = df[ df['Borough'] == 'Not assigned' ].index
notassigned_hood = df[ df['Neighborhood'] == 'Not assigned' ].index

# Delete these row indexes from dataFrame
df.drop(notassigned_boro, inplace=True)
df.drop(notassigned_hood, inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Just to be sure I properly cleaned up the duplicate post codes, I checked the M5A code to be sure the data was merged as expected.

In [8]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
#Double check I've properly merged the common postal codes
df.loc[df['Postcode'] == "M5A"]

,Postcode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


**In step 2:  I took was to replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.**

Given that the Geocoder Python package can be very unreliable, in the interest of time and learning, I chose the option to use the following link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [9]:
csv_file = 'https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv'
geo_df =  pd.read_csv(csv_file)
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Next step was to combine this data with our neighborhood dataframe.

In [10]:

toronto_df = df.join(geo_df)
toronto_df.drop(['Postal Code'], axis=1, inplace=True)

toronto_df = toronto_df.groupby(['Postcode','Borough', 'Latitude', 'Longitude'])['Neighborhood'].apply(', '.join).reset_index()


toronto_df

,Postcode,Borough,Latitude,Longitude,Neighborhood
0,M1B,Scarborough,43.806686,-79.194353,"Rouge, Malvern"
1,M1C,Scarborough,43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,43.763573,-79.188711,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,43.770992,-79.216917,Woburn
4,M1H,Scarborough,43.773136,-79.239476,Cedarbrae
5,M1J,Scarborough,43.744734,-79.239476,Scarborough Village
6,M1K,Scarborough,43.727929,-79.262029,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,43.711112,-79.284577,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,43.716316,-79.239476,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,43.692657,-79.264848,"Birch Cliff, Cliffside West"


**For the 3rd and final step, I'm going to analyze the neighborhoods using Foursquare database and discuss any interesting findings.**

Import the necessary libraries

In [11]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Define My Foursquare Credentials

In [12]:
CLIENT_ID = 'PIOEJWDDQKDITPZIHTXAVLVGHTL5VGFTKBPLIZY5GCWWSSXL' # your Foursquare ID
CLIENT_SECRET = 'PG4RURUEQMV5XIE32HXNFPTZIFWEGEHVQZ01FOKF4NK00WZ5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PIOEJWDDQKDITPZIHTXAVLVGHTL5VGFTKBPLIZY5GCWWSSXL
CLIENT_SECRET:PG4RURUEQMV5XIE32HXNFPTZIFWEGEHVQZ01FOKF4NK00WZ5


In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Let's find the types of venues that exist in these neighborhoods

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [17]:

toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",3,3,3,3,3,3
"Alderwood, Long Branch",14,14,14,14,14,14
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,30,30,30,30,30,30


In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = list(toronto_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Neighborhood')))

# use ix to reorder
toronto_onehot = toronto_onehot.loc[:, cols]


toronto_onehot.head()



,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,...,0.0,0.033333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.055556,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.033333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [20]:
toronto_grouped.shape

(99, 233)

 Let's print each neighborhood along with the top 5 most common venues

In [21]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0              Café  0.10
1             Hotel  0.07
2        Steakhouse  0.07
3  Asian Restaurant  0.07
4       Pizza Place  0.03


----Agincourt----
               venue  freq
0     Breakfast Spot  0.25
1             Lounge  0.25
2     Clothing Store  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0               Park  0.67
1         Playground  0.33
2  Accessories Store  0.00
3             Museum  0.00
4     Medical Center  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
               venue  freq
0        Pizza Place  0.33
1               Park  0.33
2           Bus Line  0.33
3  Accessories Store  0.00
4             Museum  0.00


----Alderwood, Long Branch----
            venue  freq
0            Café  0.14
1             Gym  0.07
2          Ba

First, let's write a function to sort the venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [23]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Hotel,Steakhouse,Asian Restaurant,Monument / Landmark,Lounge,Smoke Shop,Noodle House,Speakeasy,Japanese Restaurant
1,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Park,Bus Line,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
4,"Alderwood, Long Branch",Café,Liquor Store,Fried Chicken Joint,Pizza Place,Gym,Restaurant,Sandwich Place,Fast Food Restaurant,Flower Shop,Bakery


**Finally, let's begin clustering our data using the sklearn library for KMeans...**

In [24]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 2, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [25]:
neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Hotel,Steakhouse,Asian Restaurant,Monument / Landmark,Lounge,Smoke Shop,Noodle House,Speakeasy,Japanese Restaurant
1,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Park,Bus Line,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
4,"Alderwood, Long Branch",Café,Liquor Store,Fried Chicken Joint,Pizza Place,Gym,Restaurant,Sandwich Place,Fast Food Restaurant,Flower Shop,Bakery


In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!




,Postcode,Borough,Latitude,Longitude,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,43.806686,-79.194353,"Rouge, Malvern",4.0,Fast Food Restaurant,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,M1C,Scarborough,43.784535,-79.160497,"Highland Creek, Rouge Hill, Port Union",0.0,Bar,Yoga Studio,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,M1E,Scarborough,43.763573,-79.188711,"Guildwood, Morningside, West Hill",1.0,Pizza Place,Breakfast Spot,Medical Center,Electronics Store,Mexican Restaurant,Rental Car Location,Intersection,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant
3,M1G,Scarborough,43.770992,-79.216917,Woburn,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
4,M1H,Scarborough,43.773136,-79.239476,Cedarbrae,1.0,Bakery,Bank,Hakka Restaurant,Thai Restaurant,Caribbean Restaurant,Athletics & Sports,Fried Chicken Joint,Dog Run,Discount Store,Yoga Studio


In [27]:
#Clean up dataframe for NaN type errors.
toronto_merged = toronto_merged.dropna(how = 'any')
toronto_merged.isna().sum()

Postcode                  0
Borough                   0
Latitude                  0
Longitude                 0
Neighborhood              0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

Finally, let's visualize the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

***Examining the clusters:***      Summary of the most common venue that defines each of the 5 clusters

In [80]:

tm0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
tm1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
tm2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
tm3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
tm4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

print('Cluster 1 is:', tm0['1st Most Common Venue'].value_counts().idxmax())
print('Cluster 2 is:', tm1['1st Most Common Venue'].value_counts().idxmax())
print('Cluster 3 is:', tm2['1st Most Common Venue'].value_counts().idxmax())
print('Cluster 4 is:', tm3['1st Most Common Venue'].value_counts().idxmax())
print('Cluster 5 is:', tm4['1st Most Common Venue'].value_counts().idxmax())

Cluster 1 is: Home Service
Cluster 2 is: Café
Cluster 3 is: Park
Cluster 4 is: Martial Arts Dojo
Cluster 5 is: Fast Food Restaurant


Details behind the top 10 venues in each cluster by Neighborhood

In [72]:
print(tm0['1st Most Common Venue'].value_counts().idxmax(), 'Cluster')
tm0

Home Service Cluster


,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,"Highland Creek, Rouge Hill, Port Union",0.0,Bar,Yoga Studio,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
21,M2M,"Newtonbrook, Willowdale",0.0,Home Service,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
32,M3M,Downsview Central,0.0,Korean Restaurant,Food Truck,Home Service,Baseball Field,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
63,M5N,Roselawn,0.0,Garden,Health & Beauty Service,Home Service,Department Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
91,M8Z,"Kingsway Park South West, Mimico NW, The Queen...",0.0,Home Service,Baseball Field,Yoga Studio,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
97,M9N,Weston,0.0,Baseball Field,Yoga Studio,Dim Sum Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [76]:
print(tm1['1st Most Common Venue'].value_counts().idxmax(), 'Cluster')
tm1

Café Cluster


,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,"Guildwood, Morningside, West Hill",1.0,Pizza Place,Breakfast Spot,Medical Center,Electronics Store,Mexican Restaurant,Rental Car Location,Intersection,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant
3,M1G,Woburn,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
4,M1H,Cedarbrae,1.0,Bakery,Bank,Hakka Restaurant,Thai Restaurant,Caribbean Restaurant,Athletics & Sports,Fried Chicken Joint,Dog Run,Discount Store,Yoga Studio
5,M1J,Scarborough Village,1.0,Playground,Convenience Store,Department Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",1.0,Coffee Shop,Convenience Store,Chinese Restaurant,Discount Store,Department Store,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space
7,M1L,"Clairlea, Golden Mile, Oakridge",1.0,Bakery,Bus Line,Fast Food Restaurant,Soccer Field,Metro Station,Bus Station,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",1.0,American Restaurant,Motel,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
9,M1N,"Birch Cliff, Cliffside West",1.0,College Stadium,General Entertainment,Skating Rink,Café,Concert Hall,Construction & Landscaping,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
10,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",1.0,Indian Restaurant,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Latin American Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Eastern European Restaurant
11,M1R,"Maryvale, Wexford",1.0,Middle Eastern Restaurant,Breakfast Spot,Smoke Shop,Shopping Mall,Sandwich Place,Auto Garage,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant


In [77]:
print(tm2['1st Most Common Venue'].value_counts().idxmax(), 'Cluster')
tm2

Park Cluster


,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M1V,"Agincourt North, L'Amoreaux East, Milliken, St...",2.0,Park,Playground,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
25,M3A,Parkwoods,2.0,Food & Drink Shop,Park,Yoga Studio,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
30,M3K,"CFB Toronto, Downsview East",2.0,Airport,Park,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
40,M4J,East Toronto,2.0,Park,Pizza Place,Coffee Shop,Convenience Store,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
44,M4N,Lawrence Park,2.0,Park,Swim School,Bus Line,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
50,M4W,Rosedale,2.0,Park,Playground,Building,Trail,Deli / Bodega,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
74,M6E,Caledonia-Fairbanks,2.0,Park,Women's Store,Fast Food Restaurant,Market,Construction & Landscaping,Diner,Comic Shop,Farmers Market,Falafel Restaurant,Event Space
90,M8Y,"Humber Bay, King's Mill Park, Kingsway Park So...",2.0,Park,River,Yoga Studio,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
98,M9P,Westmount,2.0,Park,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
100,M9V,"Albion Gardens, Beaumond Heights, Humbergate, ...",2.0,Pizza Place,Park,Bus Line,Yoga Studio,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


In [78]:
print(tm3['1st Most Common Venue'].value_counts().idxmax(), 'Cluster')
tm3

Martial Arts Dojo Cluster


,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M2L,"Silver Hills, York Mills",3.0,Martial Arts Dojo,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


In [79]:
print(tm4['1st Most Common Venue'].value_counts().idxmax(), 'Cluster')
tm4

Fast Food Restaurant Cluster


,Postcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",4.0,Fast Food Restaurant,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
